In [3]:
import pandas as pd
import numpy as np
from IPython.display import display_html
from itertools import chain, cycle

df = pd.read_excel("C:/Users/Arshad/Desktop/Python/Stocks_Capital_Gains_Report_5561318264_01-04-2023_31-03-2024.xlsx")

# Find the headers
header = df.iloc[2,0]

# Platform and brokerage charges
charges=df.iloc[7:19,0:2]
charges_colname = ['Groww charges','Amount']
charges.columns = charges_colname
charges['Amount'] = charges['Amount'].astype(float)
charges.reset_index(drop=True,inplace=True)

# 3.Merge total and dividends as total gain
toth=df.index[df['Name']=='Realised P&L']
total=df.iloc[toth[0]+1:toth[0]+4,0:2]
total_colname = ['Category','Amount Gained']
total.columns = total_colname
total.reset_index(drop=True,inplace=True)
total['Amount Gained'] = total['Amount Gained'].astype(float)

others=df.iloc[toth[0]+9:toth[0]+11,0:2]
others_colname = ['Category','Amount Gained']
others.columns = others_colname
others.reset_index(drop=True,inplace=True)
others['Amount Gained'] = others['Amount Gained'].astype(float)

totgain = pd.concat([total,others],axis=0,ignore_index=True)
totgain.loc['5',:] = ['Total Charges (STT, DP, Brokerage)',charges['Amount'].sum()]
total_gain = totgain.loc[totgain['Category'] != 'Total Charges (STT, DP, Brokerage)', 'Amount Gained'].sum()
total_charges = totgain.loc[totgain['Category'] == 'Total Charges (STT, DP, Brokerage)', 'Amount Gained'].sum()
totgain.loc['6',:] = ['Net Profit/Loss',total_gain - total_charges]

# 4.Stock differentiation
shth=df.index[df['Name']=='Intraday trades']
stocks=df.iloc[shth[0]+2:,:]
empty = stocks[stocks.isnull().all(axis=1)]
emtrow=empty.index
stock_colname = ['Stock','ISIN','Quantity','Buy date','Buy price','Buy value','Sell date','Sell price','Sell value','P&L','Remarks']

# 5.Intraday stocks
intra=df.iloc[shth[0]+2:emtrow[0],:]
intra.columns = stock_colname
intra.reset_index(drop=True,inplace=True)
intra.loc[:,'Quantity'] = pd.to_numeric(intra['Quantity'], errors='coerce')
intra.loc[:,'Buy value'] = pd.to_numeric(intra['Buy value'], errors='coerce')
intra.loc[:,'Buy price']  = pd.to_numeric(intra['Buy price'], errors='coerce')
intra.loc[:,'Sell value'] = pd.to_numeric(intra['Sell value'], errors='coerce')
intra.loc[:,'Sell price'] = pd.to_numeric(intra['Sell price'], errors='coerce')
intra.loc[:,'P&L'] = pd.to_numeric(intra['P&L'], errors='coerce')

# 6.Short term stocks
shth=df.index[df['Name']=='Short Term trades']
short=df.iloc[shth[0]+2:emtrow[1],:]
short.columns = stock_colname
short.reset_index(drop=True,inplace=True)
short.loc[:,'Quantity'] = pd.to_numeric(short['Quantity'], errors='coerce')
short.loc[:,'Buy value'] = pd.to_numeric(short['Buy value'], errors='coerce')
short.loc[:,'Buy price']  = pd.to_numeric(short['Buy price'], errors='coerce')
short.loc[:,'Sell value'] = pd.to_numeric(short['Sell value'], errors='coerce')
short.loc[:,'Sell price'] = pd.to_numeric(short['Sell price'], errors='coerce')
short.loc[:,'P&L'] = pd.to_numeric(short['P&L'], errors='coerce')

# # 7.Long term stocks
lngh=df.index[df['Name']=='Long Term trades']
long=df.iloc[lngh[0]+2:emtrow[2],:]
long.columns = stock_colname
long.reset_index(drop=True,inplace=True)
long.loc[:,'Quantity'] = pd.to_numeric(long['Quantity'], errors='coerce')
long.loc[:,'Buy value'] = pd.to_numeric(long['Buy value'], errors='coerce')
long.loc[:,'Buy price']  = pd.to_numeric(long['Buy price'], errors='coerce')
long.loc[:,'Sell value'] = pd.to_numeric(long['Sell value'], errors='coerce')
long.loc[:,'Sell price'] = pd.to_numeric(long['Sell price'], errors='coerce')
long.loc[:,'P&L'] = pd.to_numeric(long['P&L'], errors='coerce')

# Analysis on Intraday
intra_summary = intra.groupby("Stock").agg({'Quantity': 'sum','Buy value': 'sum','Sell value': 'sum','P&L': 'sum'}).reset_index()
intra_summary.sort_values(by='P&L', ascending=False, inplace=True)
intra_summary['Quantity'] = pd.to_numeric(intra_summary['Quantity'], errors='coerce')
intra_summary['Buy value'] = pd.to_numeric(intra_summary['Buy value'], errors='coerce')
intra_summary['Sell value'] = pd.to_numeric(intra_summary['Sell value'], errors='coerce')
intra_summary['P&L'] = pd.to_numeric(intra_summary['P&L'], errors='coerce')

# Analysis on Short term
short_summary = short.groupby("Stock").agg({'Quantity': 'sum','Buy value': 'sum','Sell value': 'sum','P&L': 'sum'}).reset_index()
short_summary.sort_values(by='P&L', ascending=False, inplace=True)
short_summary['Quantity'] = pd.to_numeric(short_summary['Quantity'], errors='coerce')
short_summary['Buy value'] = pd.to_numeric(short_summary['Buy value'], errors='coerce')
short_summary['Sell value'] = pd.to_numeric(short_summary['Sell value'], errors='coerce')
short_summary['P&L'] = pd.to_numeric(short_summary['P&L'], errors='coerce')

# Analysis on Long term
long_summary = long.groupby("Stock").agg({'Quantity': 'sum','Buy value': 'sum','Sell value': 'sum','P&L': 'sum'}).reset_index()
long_summary.sort_values(by='P&L', ascending=False, inplace=True)
long_summary['Quantity'] = pd.to_numeric(long_summary['Quantity'], errors='coerce')
long_summary['Buy value'] = pd.to_numeric(long_summary['Buy value'], errors='coerce')
long_summary['Sell value'] = pd.to_numeric(long_summary['Sell value'], errors='coerce')
long_summary['P&L'] = pd.to_numeric(long_summary['P&L'], errors='coerce')

# Displaying dataframes together
def display_side_by_side(*args, titles=cycle([''])):
    html_str = ''
    for df, title in zip(args, chain(titles, cycle(['</br>']))):
        html_str += '<th style="text-align:center"><td style="vertical-align:top">'
        html_str += f'<h2 style="text-align: center;">{title}</h2>'
        html_str += df.to_html().replace('table', 'table style="display:inline"')
        html_str += '</td></th>'
    display_html(html_str, raw=True)
display_side_by_side(totgain, intra_summary, short_summary, long_summary, titles=['Overall Summary','Intraday Summary', 'Short Term Stock Summary', 'Long Term Stock Summary']) 

Overall Summary 
 
 
 
 Category 
 Amount Gained 
 
 
 
 
 0 
 Intraday P&L 
 -130.70 
 
 
 1 
 Short Term P&L 
 5960.77 
 
 
 2 
 Long Term P&L 
 0.00 
 
 
 3 
 Dividends 
 2219.25 
 
 
 4 
 Buyback P&L 
 0.00 
 
 
 5 
 Total Charges (STT, DP, Brokerage) 
 2223.54 
 
 
 6 
 Net Profit/Loss 
 5825.78 
 
 
 Intraday Summary 
 
 
 
 Stock 
 Quantity 
 Buy value 
 Sell value 
 P&L 
 
 
 
 
 0 
 BHEL 
 2 
 256.2 
 254.0 
 -2.2 
 
 
 1 
 INFIBEAM AVENUES LIMITED 
 420 
 7552.0 
 7423.5 
 -128.5 
 
 
 Short Term Stock Summary 
 
 
 
 Stock 
 Quantity 
 Buy value 
 Sell value 
 P&L 
 
 
 
 
 19 
 URJA GLOBAL LIMITED 
 800 
 14234.40 
 20600.86 
 6366.46 
 
 
 7 
 INDIAN OIL CORP LTD 
 250 
 24442.50 
 29362.50 
 4920.00 
 
 
 15 
 NMDC LTD. 
 135 
 25503.50 
 28865.50 
 3362.00 
 
 
 14 
 Mishtann Foods Limited 
 500 
 7758.00 
 10945.00 
 3187.00 
 
 
 2 
 BHARAT PETROLEUM CORP LT 
 25 
 9646.25 
 11571.75 
 1925.50 
 
 
 18 
 TVS MOTOR COMPANY LTD 
 2 
 3169.60 
 4253.10 
 1083.50 
 
 
 0 
 ADANI PORT & SEZ LTD 
 15 
 19094.25 
 19534.65 
 440.40 
 
 
 16 
 SPICEJET LTD. 
 60 
 2297.79 
 2646.60 
 348.81 
 
 
 3 
 CANARA BANK 
 2 
 757.80 
 1066.70 
 308.90 
 
 
 5 
 ENGINEERS INDIA LTD 
 45 
 6664.50 
 6936.75 
 272.25 
 
 
 20 
 VAKRANGEE LIMITED 
 165 
 3425.25 
 3600.30 
 175.05 
 
 
 9 
 INFIBEAM AVENUES LIMITED 
 100 
 1975.00 
 2140.00 
 165.00 
 
 
 11 
 INTERGLOBE AVIATION LTD 
 1 
 2939.80 
 3047.00 
 107.20 
 
 
 13 
 MMTC LIMITED 
 115 
 6852.00 
 6888.05 
 36.05 
 
 
 8 
 INDIAN RENEWABLE ENERGY 
 170 
 20036.00 
 19981.50 
 -54.50 
 
 
 21 
 WELSPUN LIVING LIMITED 
 45 
 6970.50 
 6832.50 
 -138.00 
 
 
 10 
 INTEGRA ESSENTIA LIMITED 
 1000 
 5170.00 
 4600.00 
 -570.00 
 
 
 12 
 LINDE INDIA LIMITED 
 1 
 6686.65 
 6104.05 
 -582.60 
 
 
 17 
 TATA TECHNOLOGIES LIMITED 
 10 
 12041.75 
 11022.20 
 -1019.55 
 
 
 6 
 GLOBAL HEALTH LIMITED 
 10 
 13499.50 
 12381.50 
 -1118.00 
 
 
 4 
 DOMS INDUSTRIES LIMITED 
 20 
 31730.70 
 29160.40 
 -2570.30 
 
 
 1 
 AVANTEL LTD. 
 965 
 104222.25 
 93537.85 
 -10684.40 
 
 
 Long Term Stock Summary 
 
 
 
 Stock 
 Quantity 
 Buy value 
 Sell value 
 P&L